In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
# Load tidied dataframes

# spells_df = pd.read_pickle('data/spells_df')
monster_df = pd.read_pickle('data/monster_df')
actions_df = pd.read_pickle('data/actions_df')
spellbook_df = pd.read_pickle('data/spellbook_df')
innate_spellbook_df = pd.read_pickle('data/innate_spellbook_df')
# special_abilities_df = pd.read_pickle('data/special_abilities_df')
# condition_immunities_df = pd.read_pickle('data/condition_immunities_df')
# damage_immunities_df = pd.read_pickle('data/damage_immunities_df')
# damage_resistances_df = pd.read_pickle('data/damage_resistances_df')
# damage_vulnerabilities_df = pd.read_pickle('data/damage_vulnerabilities_df')

# `actions_df`

In [3]:
actions_df = actions_df.reset_index('action')

In [4]:
# ill-formed multiattack statuses

outliers = ['Animated Armor',
            'Cult Fanatic',
            'Deva',
            'Doppelganger',
            'Erinyes',
            'Gibbering Mouther',
            'Gladiator',
            'Iron Golem',
            'Knight',
            'Lizardfolk',
            'Planetar',
            'Scout',
            'Spy',
            'Thug',
            'Violet Fungus',
            'Werebear']

In [5]:
a = (actions_df.index).difference(outliers)
df = actions_df.loc[a]

In [6]:
# actions_df[actions_df.index.isin(outliers)]

# multiattack

In [47]:
def normalize_multiattack(x):
    df = x.set_index('action')
    if 'Multiattack' not in df.index:
        return
    desc = df.loc['Multiattack'].desc.lower().strip()
    actions = [format_action(action) for action in df.index
               if action != 'Multiattack']
    multiattack, exception = extract_attacks(desc, actions)
    return pd.DataFrame({'desc': [desc],
                         'multiattack': [multiattack],
                         'exception': [exception]})


def format_action(action):
    action = action.lower()
    i = action.find('(')
    if i > 0:
        return action[:i-1]
    else:
        return action


def extract_attacks(desc, actions):
    exceptions_pattern = r'\b({})\b'.format('|'.join(x for x in exceptions))
    exception = re.search(exceptions_pattern, desc) is not None
    actions_pattern = '|'.join(x for x in actions)
    split_pattern = r'.*?({})'.format(actions_pattern)
    numerics_pattern = '|'.join(x for x in numerics.keys())
    multiattack = list()
    for x in re.finditer(split_pattern, desc):
        start, end = x.span()
        x = re.findall(numerics_pattern, desc[start:end])
        if x:
            n = numerics[x[-1]]
        else:
            n = 1
        y = re.findall(actions_pattern, desc[start:end])
        multiattack.extend(n * y)
    return multiattack, exception


numerics = {'one': 1,
            'two': 2,
            'twice': 2,
            'three': 3,
            'four': 4,
            'six': 6,
            'seven': 7}
exceptions = ('or', 'alternatively', 'when', 'in place', 'replace', 'also', 'as many', 'only')

In [48]:
multiattacks_df = df.groupby('name').apply(normalize_multiattack)
multiattacks_df.reset_index(level=1, drop=True, inplace=True)

We'll correct the following by hand

In [49]:
multiattacks_df[multiattacks_df.exception]

,desc,exception,multiattack
name,,,
Bandit Captain,the captain makes three melee attacks: two wit...,True,"[scimitar, scimitar, dagger, dagger, dagger]"
Barbed Devil,the devil makes three melee attacks: one with ...,True,"[tail, claw, claw, hurl flame]"
Centaur,the centaur makes two attacks: one with its pi...,True,"[pike, hooves, longbow, longbow]"
Chimera,the chimera makes three attacks: one with its ...,True,"[bite, horns, claws, fire breath, bite, horns]"
Chuul,the chuul makes two pincer attacks. if the chu...,True,"[pincer, pincer, tentacles]"
Dragon Turtle,the dragon turtle makes three attacks: one wit...,True,"[bite, claw, claw, tail, claw, claw]"
Drider,"the drider makes three attacks, either with it...",True,"[longsword, longsword, longsword, longbow, bite]"
Efreeti,the efreeti makes two scimitar attacks or uses...,True,"[scimitar, scimitar, hurl flame]"
Glabrezu,the glabrezu makes four attacks: two with its ...,True,"[pincer, pincer, fist, fist, pincer, pincer]"


There are lots of special exceptions to deal with?

# other actions

In [60]:
def damage_summary(damage_dice, damage_bonus):
    pattern = re.compile(r'(\d+)d(\d+)')
    damage = [np.array([damage_bonus, 0])]
    for dice in re.finditer(pattern, damage_dice):
        x, y = dice.groups()
        damage.append(damage_statistics(int(x), int(y)))
    mean, variance = sum(damage)
    return mean, variance


def damage_statistics(number, size):
    mu = (size + 1) / 2
    mean = number * mu
    var = number * (sum_of_squares(size) / size - mu ** 2)
    return np.array([mean, var])


def sum_of_squares(n):
    return n*(2*n+1)*(n+1)/6

In [89]:
actions_df[actions_df.action == 'Bite'].loc['Adult Blue Dracolich'].desc

'Melee Weapon Attack: +12 to hit, reach 10 ft., one target. Hit: 18 (2d10 + 7) piercing damage plus 5 (1d10) lightning damage.'

Some of the breath weapons have to be split up.

In [112]:
desc = 'Melee Weapon Attack: +12 to hit, reach 15 ft., one target. Hit: 16 (2d8 + 7) bludgeoning damage.'
pattern = re.compile(r'^(.* Attack: .* targets?). (Hit: .* damage.)')
g = re.match(pattern, desc)

In [114]:
g.span()

(0, 96)

In [150]:
def parse_desc(desc):
    pattern = re.compile(r'^(.* Attack: .* (creature|target)s?)\. (Hit: [^\.]*)')
    g = re.match(pattern, desc)
    if g:
        start, end = g.span()
        attack, _, result = g.groups()
        print (parse_attack(attack), parse_result(result))
    else:
        pass
    return

def parse_attack(attack):
    pattern = re.compile(r'Attack: ' \
                         + r'+(\d+) to hit, ' \
                         + r'(.*)ft\.\W?, ' \
                         + r''
                        )
    return attack


def parse_result(result):
    return ''


actions_df.desc.apply(parse_desc)

Melee Weapon Attack: +9 to hit, reach 10 ft., one target 
Melee Weapon Attack: +9 to hit, reach 10 ft. one target 
Melee Weapon Attack: +2 to hit, reach 5 ft., one target 
Melee Weapon Attack: +11 to hit, reach 10 ft., one target 
Melee Weapon Attack: +11 to hit, reach 5 ft., one target 
Melee Weapon Attack: +11 to hit, reach 15 ft., one target 
Melee Weapon Attack: +12 to hit, reach 10 ft., one target 
Melee Weapon Attack: +12 to hit, reach 5 ft., one target 
Melee Weapon Attack: +12 to hit, reach 15 ft., one target 
Melee Weapon Attack: +12 to hit, reach 10 ft., one target 
Melee Weapon Attack: +12 to hit, reach 5 ft., one target 
Melee Weapon Attack: +12 to hit, reach 15 ft., one target 
Melee Weapon Attack: +11 to hit, reach,.0 ft., one target 
Melee Weapon Attack: +11 to hit, reach 5 ft., one target 
Melee Weapon Attack: +11 to hit, reach 15 ft., one target 
Melee Weapon Attack: +12 to hit, reach 10 ft., one target 
Melee Weapon Attack: +12 to hit, reach 5 ft., one target 
Melee W

name
Aboleth                 None
Aboleth                 None
Aboleth                 None
Aboleth                 None
Acolyte                 None
Adult Black Dragon      None
Adult Black Dragon      None
Adult Black Dragon      None
Adult Black Dragon      None
Adult Black Dragon      None
Adult Black Dragon      None
Adult Blue Dracolich    None
Adult Blue Dracolich    None
Adult Blue Dracolich    None
Adult Blue Dracolich    None
Adult Blue Dracolich    None
Adult Blue Dracolich    None
Adult Blue Dragon       None
Adult Blue Dragon       None
Adult Blue Dragon       None
Adult Blue Dragon       None
Adult Blue Dragon       None
Adult Blue Dragon       None
Adult Brass Dragon      None
Adult Brass Dragon      None
Adult Brass Dragon      None
Adult Brass Dragon      None
Adult Brass Dragon      None
Adult Brass Dragon      None
Adult Bronze Dragon     None
                        ... 
Young Brass Dragon      None
Young Bronze Dragon     None
Young Bronze Dragon     None
Young Bro

In [129]:

for x in actions_df.iterrows():
    if x[1].action == 'Multiattack':
        continue
    if not x[1].damage_dice:
        continue
    print('{} - {}'.format(x[0], x[1].action))
    mean, var = damage_summary(x[1].damage_dice,
                               x[1].damage_bonus)
    print('\n')
    # print('mean: {:0.2f}\nstd : {:0.2f}'.format(mean, np.sqrt(var)))

Aboleth - Tentacle
('9', 'reach 10', ' one', 'target', '12 (2d6 + 5) bludgeoning damage')
Melee Weapon Attack: +9 to hit, reach 10 ft., one target. Hit: 12 (2d6 + 5) bludgeoning damage. If the target is a creature, it must succeed on a DC 14 Constitution saving throw or become diseased. The disease has no effect for 1 minute and can be removed by any magic that cures disease. After 1 minute, the diseased creature's skin becomes translucent and slimy, the creature can't regain hit points unless it is underwater, and the disease can be removed only by heal or another disease-curing spell of 6th level or higher. When the creature is outside a body of water, it takes 6 (1d12) acid damage every 10 minutes unless moisture is applied to the skin before 10 minutes have passed.


Aboleth - Tail
('9', 'reach 10', ' one', 'target', '15 (3d6 + 5) bludgeoning damage')
Melee Weapon Attack: +9 to hit, reach 10 ft. one target. Hit: 15 (3d6 + 5) bludgeoning damage.


Acolyte - Club
('2', 'reach 5', ' o



Cultist - Scimitar
('3', 'reach 5', ' one', 'creature', '4 (1d6 + 1) slashing damage')
Melee Weapon Attack: +3 to hit, reach 5 ft., one creature. Hit: 4 (1d6 + 1) slashing damage.


Darkmantle - Crush
('5', 'reach 5', ' one', 'creature', '6 (1d6 + 3) bludgeoning damage, and the darkmantle attaches to the target')
Melee Weapon Attack: +5 to hit, reach 5 ft., one creature. Hit: 6 (1d6 + 3) bludgeoning damage, and the darkmantle attaches to the target. If the target is Medium or smaller and the darkmantle has advantage on the attack roll, it attaches by engulfing the target's head, and the target is also blinded and unable to breathe while the darkmantle is attached in this way.
While attached to the target, the darkmantle can attack no other creature except the target but has advantage on its attack rolls. The darkmantle's speed also becomes 0, it can't benefit from any bonus to its speed, and it moves with the target.
A creature can detach the darkmantle by making a successful DC 13 S

In [121]:
oddities = ('Adult Brass Dragon - Bite',
            'Ancient Black Dragon - Tail')

In [61]:
damage_summary('1d6 + 2d4', 4)

(12.5, 5.4166666666666661)

In [42]:
actions_df.damage_dice.apply(parse_damage_dice)

name
Aboleth                                 []
Aboleth                           [(2, 6)]
Aboleth                           [(3, 6)]
Aboleth                                 []
Acolyte                           [(1, 4)]
Adult Black Dragon                      []
Adult Black Dragon       [(2, 10), (1, 8)]
Adult Black Dragon                [(2, 6)]
Adult Black Dragon                [(2, 8)]
Adult Black Dragon                      []
Adult Black Dragon               [(12, 8)]
Adult Blue Dracolich                    []
Adult Blue Dracolich    [(2, 10), (1, 10)]
Adult Blue Dracolich              [(2, 6)]
Adult Blue Dracolich              [(2, 8)]
Adult Blue Dracolich                    []
Adult Blue Dracolich            [(12, 10)]
Adult Blue Dragon                       []
Adult Blue Dragon       [(2, 10), (1, 10)]
Adult Blue Dragon                 [(2, 6)]
Adult Blue Dragon                 [(2, 8)]
Adult Blue Dragon                       []
Adult Blue Dragon               [(12, 10)]
Adult 